In [1]:
import pandas as pd

# stats to calculate
- ## outsourcing:
  - percentage of books scanned at BPOs per year
- ## pay
    - calculations on maximum wages workers at cebu could have made based on number of workers there per month and the contract IA had with Innodata at the time (at 1.5million per year / 12 months with x workers working per month, that leaves what max monthly wage).
- ## rates of work:
  - as measure of pages scanned to workers working per center per month
  - avergage pages scanned to workers working per center + standard deviation
- ## turn over rates- how long do operators stay at their jobs across centers
  - avg. days worked by workers at each center + standard deviation - will need to throw out lots of junk

In [2]:
# accessing the geocoded texts dataset from my external hard drive. You can get it from this box link: 
texts = pd.read_csv('/Volumes/Samsung_T5/scanning_labor_in_IA/geocoded-texts-data.csv', low_memory=False)
texts = texts.drop(columns=['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'])

In [39]:
# scans per center per month
scans_center_month = pd.read_csv("https://raw.githubusercontent.com/ers6/ia_scanning_labor_data/main/metadata-analysis/metadata-records-analysis-csvs/scans_per_center_per_month.csv") 

# pages per center per month
pages_center_month = pd.read_csv("https://raw.githubusercontent.com/ers6/ia_scanning_labor_data/main/metadata-analysis/metadata-records-analysis-csvs/pages_per_center_per_month.csv")

# scans per center type per month
scans_type_month = pd.read_csv("https://raw.githubusercontent.com/ers6/ia_scanning_labor_data/main/metadata-analysis/metadata-records-analysis-csvs/scans_per_center_type_per_month.csv").dropna(subset=['center_type'])
scans_type_month = scans_type_month[['month_year','center_type', 'books_scanned']]


In [44]:
scans_type_year = scans_type_month.groupby(['center_type','month_year']).sum().reset_index()

In [45]:
scans_type_year['year'] = ""

for i in range(len(scans_type_year)):
    scans_type_year.at[i, 'year'] = str(scans_type_year.at[i,'month_year']).split('-')[0]

In [46]:
scans_type_year = scans_type_year.groupby(['center_type','year']).sum().reset_index()[['center_type', 'year', 'books_scanned']]

In [47]:
scans_type_year

,center_type,year,books_scanned
0,academic,2001,3
1,academic,2002,421
2,academic,2006,37248
3,academic,2007,108590
4,academic,2008,164714
...,...,...,...
118,public,2019,21036
119,public,2020,8342
120,public,2021,16017
121,public,2022,29775


In [76]:
scans_per_year = scans_type_year.groupby(['year']).sum().reset_index()[['year','books_scanned']]



scans_per_year.loc[scans_per_year['year']== '2010']['books_scanned'].tolist()[0]

261573

In [81]:
scans_type_year['percent'] = ''

for i in range(len(scans_type_year)):
    scans_type_year.at[i, 'percent'] = scans_type_year.at[i, 'books_scanned'] / scans_per_year.loc[scans_per_year['year'] == scans_type_year.at[i, 'year']]['books_scanned'].tolist()[0]

 

In [82]:
scans_type_year.to_csv('')

,center_type,year,books_scanned,percent
0,academic,2001,3,1.0
1,academic,2002,421,0.97907
2,academic,2006,37248,0.589554
3,academic,2007,108590,0.683085
4,academic,2008,164714,0.639723
...,...,...,...,...
118,public,2019,21036,0.043464
119,public,2020,8342,0.010275
120,public,2021,16017,0.016478
121,public,2022,29775,0.027982
